In [1]:
import json
import pandas as pd
from scale import *

### Processing Contractors

In [2]:
df = pd.read_csv('Contractors.csv')

In [3]:
res = []

for i in list(df.index):
    temp = {}
    for e in list(df.columns):
        if e != 'LAT' and e != 'LONG' and e != 'Contractor':
            temp[e] = int(df.loc[i, e])
        else:
            temp[e] = df.loc[i,e]
    res.append(temp)

In [4]:
with open("Contractors.json", 'w') as outfile:
    json.dump(res, outfile)

### Processing Urban Population

In [2]:
df = pd.read_csv('CAUrbanAreaPopulation.csv')
with open("CAUrbanArea.json") as f:
    data = json.load(f)

In [3]:
# merge based on name
for i in range(len(data['features'])):
    temp = data['features'][i]['properties']['NAME10'].split(',')[0].strip()
    dftemp = df.loc[df['NAME'] == temp, 'POP'].values
    if len(dftemp) > 0:
        data['features'][i]['properties']['population'] = int(dftemp[0])
        if dftemp[0] == 0:
            print(temp)

In [4]:
with open("CAUrbanArea.json", 'w') as outfile:
    json.dump(data, outfile)

### Preprocessing Powerplants

In [24]:
df = pd.read_csv('CAPowerPlants.csv')

In [25]:
df = df[(df['General_Fuel'] == 'Hydro') & (df['State_'] == 'CA') & (df['Status'] == 'Operating')].reset_index(drop = True)

In [26]:
results = []

for i in list(df.index):
    temp = {}
    temp['Plant_ID'] = df.loc[i, 'Plant_ID']
    temp['Name'] = df.loc[i, 'Plant_Label']
    temp['MW'] = df.loc[i, 'MW']
    temp['latitude'] = df.loc[i, 'Latitude']
    temp['longitude'] = df.loc[i, 'Longitude']
    results.append(temp)

In [27]:
with open("CAPowerPlants.json", 'w') as outfile:
    json.dump(results, outfile)

### Preprocessing Service Areas

In [5]:
with open("ServiceAreas.json") as f:
    data = json.load(f)

In [6]:
df = pd.read_csv('ServiceAreasInformation.csv')

In [7]:
# merge based on gid
for i in range(len(data['features'])):
    temp = data['features'][i]['properties']['gid']
    dftemp = df.loc[df['GID'] == temp, 'D_POPULATI'].values
    dftemp2 = df.loc[df['GID'] == temp, 'D_PRIN_CNT'].values
    if len(dftemp) > 0 and len(dftemp2) > 0:
        data['features'][i]['properties']['d_populati'] = int(dftemp[0])
        data['features'][i]['properties']['d_prin_cnt'] = str(dftemp2[0])
    elif len(dftemp) > 0:
        data['features'][i]['properties']['d_populati'] = int(dftemp[0])
        data['features'][i]['properties']['d_prin_cnt'] = str(dftemp2[0])
    elif len(dftemp2) > 0:
        data['features'][i]['properties']['d_populati'] = 0 
        data['features'][i]['properties']['d_prin_cnt'] = str(dftemp2[0])
    else:
        data['features'][i]['properties']['d_populati'] = 0
        data['features'][i]['properties']['d_prin_cnt'] = str("")

In [8]:
# total population
import random
random.seed(1)

master_pop = {}
for i in range(len(data['features'])):
    if data['features'][i]['properties']['d_prin_cnt'] in master_pop:
        master_pop[data['features'][i]['properties']['d_prin_cnt']] = data['features'][i]['properties']['d_populati'] + master_pop[data['features'][i]['properties']['d_prin_cnt']]
    else:
        master_pop[data['features'][i]['properties']['d_prin_cnt']] = data['features'][i]['properties']['d_populati']
        #master_color[data['features'][i]['properties']['d_prin_cnt']] = "#" + str(random.random())[2:8]

In [9]:
master_color = {}
values = list(master_pop.values())

color = LinearScale(domain=[min(values), max(values)], range=["#00ff40","#0f5922"])
for idx, e in enumerate(master_pop):
    master_color[e] = color(values[idx])

In [10]:
for i in range(len(data['features'])):
    data['features'][i]['properties']['total_population'] = master_pop[data['features'][i]['properties']['d_prin_cnt']]
    data['features'][i]['properties']['color'] = str(master_color[data['features'][i]['properties']['d_prin_cnt']])

In [11]:
with open("ServiceAreasTemp.json", 'w') as outfile:
    json.dump(data, outfile)